In [81]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import nltk

from textblob import TextBlob

import numpy as np
import pandas as pd

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')

nltk.download('punkt')

from nltk.stem import PorterStemmer

from textblob import Word
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mxnog\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mxnog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mxnog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
train=pd.read_csv("Corona_NLP_train.csv",encoding='latin1')
test=pd.read_csv("Corona_NLP_test.csv",encoding='latin1')
train

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [5]:
train['word_count']=train['OriginalTweet'].apply(lambda x: len(str(x).split(" ")))
train[['OriginalTweet', 'word_count']].head(10)

,OriginalTweet,word_count
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,8
1,advice Talk to your neighbours family to excha...,38
2,Coronavirus Australia: Woolworths to give elde...,14
3,My food stock is not the only one which is emp...,40
4,"Me, ready to go at supermarket during the #COV...",38
5,As news of the regionÂs first confirmed COVID...,36
6,Cashier at grocery store was sharing his insig...,28
7,Was at the supermarket today. Didn't buy toile...,12
8,Due to COVID-19 our retail store and classroom...,46
9,"For corona prevention,we should stop to buy th...",42


In [6]:
train['char_count']=train['OriginalTweet'].str.len()
train[['OriginalTweet', 'char_count']].head(10)

,OriginalTweet,char_count
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,111
1,advice Talk to your neighbours family to excha...,237
2,Coronavirus Australia: Woolworths to give elde...,131
3,My food stock is not the only one which is emp...,306
4,"Me, ready to go at supermarket during the #COV...",310
5,As news of the regionÂs first confirmed COVID...,250
6,Cashier at grocery store was sharing his insig...,180
7,Was at the supermarket today. Didn't buy toile...,119
8,Due to COVID-19 our retail store and classroom...,280
9,"For corona prevention,we should stop to buy th...",267


In [7]:
def avg_word(sentence):
    words=sentence.split()
    return (sum(len(word) for word in words)/len(words))
train['avg_words'] = train['OriginalTweet'].apply(lambda x:avg_word(x))
train[['OriginalTweet', 'avg_words']].head(10)

,OriginalTweet,avg_words
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,13.000000
1,advice Talk to your neighbours family to excha...,5.263158
2,Coronavirus Australia: Woolworths to give elde...,8.428571
3,My food stock is not the only one which is emp...,6.000000
4,"Me, ready to go at supermarket during the #COV...",6.525000
5,As news of the regionÂs first confirmed COVID...,5.972222
6,Cashier at grocery store was sharing his insig...,5.464286
7,Was at the supermarket today. Didn't buy toile...,7.846154
8,Due to COVID-19 our retail store and classroom...,5.222222
9,"For corona prevention,we should stop to buy th...",5.380952


In [8]:
train['stopwords']=train['OriginalTweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['OriginalTweet', 'stopwords']].head(10)

,OriginalTweet,stopwords
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,2
1,advice Talk to your neighbours family to excha...,11
2,Coronavirus Australia: Woolworths to give elde...,1
3,My food stock is not the only one which is emp...,14
4,"Me, ready to go at supermarket during the #COV...",13
5,As news of the regionÂs first confirmed COVID...,8
6,Cashier at grocery store was sharing his insig...,9
7,Was at the supermarket today. Didn't buy toile...,2
8,Due to COVID-19 our retail store and classroom...,18
9,"For corona prevention,we should stop to buy th...",16


In [9]:
specialPatt=r"[^a-zA-Z0-9 ]"
train['special_characters']=train['OriginalTweet'].str.findall(specialPatt)
train['special_characters']=train['special_characters'].apply(lambda x: len(x))
train[['OriginalTweet', 'special_characters']].head(10)

,OriginalTweet,special_characters
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,19
1,advice Talk to your neighbours family to excha...,0
2,Coronavirus Australia: Woolworths to give elde...,8
3,My food stock is not the only one which is emp...,37
4,"Me, ready to go at supermarket during the #COV...",35
5,As news of the regionÂs first confirmed COVID...,15
6,Cashier at grocery store was sharing his insig...,12
7,Was at the supermarket today. Didn't buy toile...,18
8,Due to COVID-19 our retail store and classroom...,12
9,"For corona prevention,we should stop to buy th...",8


In [10]:
train['numerics'] = train['OriginalTweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['OriginalTweet', 'numerics']].head(10)

,OriginalTweet,numerics
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,0
1,advice Talk to your neighbours family to excha...,0
2,Coronavirus Australia: Woolworths to give elde...,0
3,My food stock is not the only one which is emp...,0
4,"Me, ready to go at supermarket during the #COV...",0
5,As news of the regionÂs first confirmed COVID...,0
6,Cashier at grocery store was sharing his insig...,0
7,Was at the supermarket today. Didn't buy toile...,0
8,Due to COVID-19 our retail store and classroom...,0
9,"For corona prevention,we should stop to buy th...",0


In [11]:
train['upper_count'] = train['OriginalTweet'].apply(lambda x: len([x for x in x.split() if x.isupper]))
train[['OriginalTweet', 'upper_count']].head(10)

,OriginalTweet,upper_count
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,8
1,advice Talk to your neighbours family to excha...,38
2,Coronavirus Australia: Woolworths to give elde...,14
3,My food stock is not the only one which is emp...,42
4,"Me, ready to go at supermarket during the #COV...",40
5,As news of the regionÂs first confirmed COVID...,36
6,Cashier at grocery store was sharing his insig...,28
7,Was at the supermarket today. Didn't buy toile...,13
8,Due to COVID-19 our retail store and classroom...,45
9,"For corona prevention,we should stop to buy th...",42


In [12]:
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https://t.co/i...
1    advice talk to your neighbours family to excha...
2    coronavirus australia: woolworths to give elde...
3    my food stock is not the only one which is emp...
4    me, ready to go at supermarket during the #cov...
5    as news of the regionâs first confirmed covid...
6    cashier at grocery store was sharing his insig...
7    was at the supermarket today. didn't buy toile...
8    due to covid-19 our retail store and classroom...
9    for corona prevention,we should stop to buy th...
Name: OriginalTweet, dtype: object

In [13]:
train['OriginalTweet'] = train['OriginalTweet'].str.replace(r"(\.|\,|!|\?|:|;)", '')
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https//tco/ifz...
1    advice talk to your neighbours family to excha...
2    coronavirus australia woolworths to give elder...
3    my food stock is not the only one which is emp...
4    me ready to go at supermarket during the #covi...
5    as news of the regionâs first confirmed covid...
6    cashier at grocery store was sharing his insig...
7    was at the supermarket today didn't buy toilet...
8    due to covid-19 our retail store and classroom...
9    for corona preventionwe should stop to buy thi...
Name: OriginalTweet, dtype: object

In [14]:
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https//tco/ifz...
1    advice talk neighbours family exchange phone n...
2    coronavirus australia woolworths give elderly ...
3    food stock one empty please panic enough food ...
4    ready go supermarket #covid19 outbreak i'm par...
5    news regionâs first confirmed covid-19 case c...
6    cashier grocery store sharing insights #covid_...
7    supermarket today buy toilet paper #rebel #toi...
8    due covid-19 retail store classroom atlanta op...
9    corona preventionwe stop buy things cash use o...
Name: OriginalTweet, dtype: object

In [15]:
freq=pd.Series(' '.join(train['OriginalTweet']).split()).value_counts()[:10]
freq

#coronavirus    15816
prices           7728
covid-19         7235
food             6698
store            6688
supermarket      6557
grocery          6023
#covid19         5613
people           5436
consumer         4214
dtype: int64

In [16]:
freq=list(freq.index)
train['OriginalTweet']= train['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https//tco/ifz...
1    advice talk neighbours family exchange phone n...
2    coronavirus australia woolworths give elderly ...
3    stock one empty please panic enough everyone t...
4    ready go outbreak i'm paranoid stock litteraly...
5    news regionâs first confirmed case came sulli...
6    cashier sharing insights #covid_19 prove credi...
7    today buy toilet paper #rebel #toiletpapercris...
8    due retail classroom atlanta open walk-in busi...
9    corona preventionwe stop buy things cash use o...
Name: OriginalTweet, dtype: object

In [17]:
leastFreq=pd.Series(' '.join(train['OriginalTweet']).split()).value_counts()[-10:]
leastFreq

https//tco/bvpcxzu9qt    1
millennium               1
https//tco/fmffoib5hj    1
https//tco/uneyyzepow    1
lockdown&ampkeeping      1
@arfonwilliams4          1
#peopleofwalmart         1
self-protection          1
https//tco/4vmunphzrw    1
infarmâs                1
dtype: int64

In [18]:
leastFreq= list(leastFreq.index)
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: " ".join(x for x in x.split() if x not in leastFreq))
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https//tco/ifz...
1    advice talk neighbours family exchange phone n...
2    coronavirus australia woolworths give elderly ...
3    stock one empty please panic enough everyone t...
4    ready go outbreak i'm paranoid stock litteraly...
5    news regionâs first confirmed case came sulli...
6    cashier sharing insights #covid_19 prove credi...
7    today buy toilet paper #rebel #toiletpapercris...
8    due retail classroom atlanta open walk-in busi...
9    corona preventionwe stop buy things cash use o...
Name: OriginalTweet, dtype: object

In [19]:
train['OriginalTweet'][:5].apply(lambda x: str(TextBlob(x).correct()))
#ZMIENIĆ NA WSZYSTKIE

0    @menyrbie @phil_gahan @christ http//to/ifz9fan...
1    advice talk neighbours family exchange phone n...
2    coronavirus australia woolworths give elderly ...
3    stock one empty please panic enough everyone t...
4    ready go outbreak i'm paranoid stock literary ...
Name: OriginalTweet, dtype: object

In [20]:
TextBlob(train['OriginalTweet'][1]).words
#ZMIENIĆ NA WSZYSTKIE

WordList(['advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'online', 'shopping', 'accounts', 'poss', 'adequate', 'supplies', 'regular', 'meds', 'order'])

In [21]:
# st = PorterStemmer()
# train['OriginalTweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
# #ZMIENIĆ NA WSZYSTKIE

In [22]:
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
train['OriginalTweet'].head(10)

0    @menyrbie @phil_gahan @chrisitv https//tco/ifz...
1    advice talk neighbour family exchange phone nu...
2    coronavirus australia woolworth give elderly d...
3    stock one empty please panic enough everyone t...
4    ready go outbreak i'm paranoid stock litteraly...
5    news regionâs first confirmed case came sulli...
6    cashier sharing insight #covid_19 prove credib...
7    today buy toilet paper #rebel #toiletpapercris...
8    due retail classroom atlanta open walk-in busi...
9    corona preventionwe stop buy thing cash use on...
Name: OriginalTweet, dtype: object

In [24]:
TextBlob(train['OriginalTweet'][1]).ngrams(2)
#zmienić na wszystkie

[WordList(['advice', 'talk']),
 WordList(['talk', 'neighbour']),
 WordList(['neighbour', 'family']),
 WordList(['family', 'exchange']),
 WordList(['exchange', 'phone']),
 WordList(['phone', 'number']),
 WordList(['number', 'create']),
 WordList(['create', 'contact']),
 WordList(['contact', 'list']),
 WordList(['list', 'phone']),
 WordList(['phone', 'number']),
 WordList(['number', 'neighbour']),
 WordList(['neighbour', 'school']),
 WordList(['school', 'employer']),
 WordList(['employer', 'chemist']),
 WordList(['chemist', 'gp']),
 WordList(['gp', 'set']),
 WordList(['set', 'online']),
 WordList(['online', 'shopping']),
 WordList(['shopping', 'account']),
 WordList(['account', 'po']),
 WordList(['po', 'adequate']),
 WordList(['adequate', 'supply']),
 WordList(['supply', 'regular']),
 WordList(['regular', 'med']),
 WordList(['med', 'order'])]

In [35]:
tf1 = (train['OriginalTweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,number,2
1,phone,2
2,neighbour,2
3,school,1
4,adequate,1
5,advice,1
6,online,1
7,regular,1
8,family,1
9,talk,1


In [37]:
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['OriginalTweet'].str.contains(word)])))
tf1

,words,tf,idf
0,number,2,4.656442
1,phone,2,5.076073
2,neighbour,2,6.060801
3,school,1,4.796204
4,adequate,1,6.733329
5,advice,1,4.997528
6,online,1,2.525595
7,regular,1,5.336882
8,family,1,3.846364
9,talk,1,4.613882


In [38]:
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,number,2,4.656442,9.312883
1,phone,2,5.076073,10.152146
2,neighbour,2,6.060801,12.121602
3,school,1,4.796204,4.796204
4,adequate,1,6.733329,6.733329
5,advice,1,4.997528,4.997528
6,online,1,2.525595,2.525595
7,regular,1,5.336882,5.336882
8,family,1,3.846364,3.846364
9,talk,1,4.613882,4.613882


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))
train_vect = tfidf.fit_transform(train['OriginalTweet'])

train_vect

<41157x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 394475 stored elements in Compressed Sparse Row format>

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(train['OriginalTweet'])
train_bow

<41157x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 439485 stored elements in Compressed Sparse Row format>

In [43]:
train['sentiment'] = train['OriginalTweet'].apply(lambda x: TextBlob(x).sentiment[0])
train[['OriginalTweet', 'sentiment']].head()

,OriginalTweet,sentiment
0,@menyrbie @phil_gahan @chrisitv https//tco/ifz...,0.000000
1,advice talk neighbour family exchange phone nu...,0.166667
2,coronavirus australia woolworth give elderly d...,-0.200000
3,stock one empty please panic enough everyone t...,0.175000
4,ready go outbreak i'm paranoid stock litteraly...,-0.077778


In [78]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

# wektoryzacja

In [79]:
from gensim.models import KeyedVectors # load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [71]:
tweets=np.array(train['OriginalTweet'])
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(tweets)

vectorizer.get_feature_names()

['00',
 '000',
 '0000',
 '0000009375',
 '000042',
 '0000hrs',
 '000162',
 '0006',
 '000francs',
 '000s',
 '001',
 '0010',
 '0016',
 '0017â',
 '0025450557',
 '0026691661',
 '005',
 '00541296',
 '009p',
 '00am',
 '00ctnbm22s',
 '00f7gzyfom',
 '00pdsup4wb',
 '00pm',
 '00pwksjyey',
 '00pxujzhmf',
 '00ry5yh8ro',
 '00urjfxstc',
 '00zyjjmcet',
 '01',
 '010',
 '0100am',
 '0101',
 '0103641972',
 '0113',
 '0115',
 '012020',
 '0121',
 '01236',
 '01273',
 '0131',
 '01392576476',
 '014',
 '01449',
 '0147',
 '01474',
 '01625',
 '0169061211',
 '017569',
 '01756986',
 '0175698634',
 '01765',
 '01892',
 '01952',
 '01ae5shygg',
 '01he1moz2x',
 '01hr',
 '01kg',
 '01laovtqn2',
 '01oc3pig5b',
 '01wbjdrnfd',
 '02',
 '020',
 '0200hrs',
 '0203',
 '02037386000',
 '0207pm',
 '021',
 '0214996028',
 '02268443322',
 '023',
 '024p',
 '025',
 '0258',
 '027',
 '0274',
 '028',
 '02890391225',
 '029',
 '02920',
 '02ch7r0rlu',
 '02ddkwsnxo',
 '02dlx6wbt3',
 '02kklx47ez',
 '02uqpatpn5',
 '02utqkozdg',
 '02zy7g1xsx',
 '03

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)
accuracyArr.append(accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)
accuracyArr.append(accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)
accuracyArr.append(accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)
accuracyArr.append(accuracy)

In [72]:
model['go']

array([-0.078894,  0.4616  ,  0.57779 , -0.71637 , -0.13121 ,  0.4186  ,
       -0.29156 ,  0.52006 ,  0.089986, -0.35062 ,  0.51755 ,  0.51998 ,
        0.15218 ,  0.41485 , -0.12377 , -0.37222 ,  0.0273  ,  0.75673 ,
       -0.8739  ,  0.58935 ,  0.46662 ,  0.62918 ,  0.092603, -0.012868,
       -0.015169,  0.25567 , -0.43025 , -0.77668 ,  0.71449 , -0.3834  ,
       -0.69638 ,  0.23522 ,  0.11396 ,  0.02778 ,  0.071357,  0.87409 ,
       -0.1281  ,  0.063576,  0.067867, -0.50181 , -0.28523 , -0.072536,
       -0.50738 , -0.6914  , -0.53579 , -0.11361 , -0.38234 , -0.12414 ,
        0.011214, -1.1622  ,  0.037057, -0.18495 ,  0.01416 ,  0.87193 ,
       -0.097309, -2.3565  , -0.14554 ,  0.28275 ,  2.0053  ,  0.23439 ,
       -0.38298 ,  0.69539 , -0.44916 , -0.094157,  0.90527 ,  0.65764 ,
        0.27628 ,  0.30688 , -0.57781 , -0.22987 , -0.083043, -0.57236 ,
       -0.299   , -0.81112 ,  0.039752, -0.05681 , -0.48879 , -0.18091 ,
       -0.28152 , -0.20559 ,  0.4932  , -0.033999, 

# koniec wektoryzacji

In [92]:
# split the dataset into training and validation datasets 
#train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train['OriginalTweet'], test['OriginalTweet'])
train_x, train_y= model_selection.train_test_split(trainDF['OriginalTweet'])
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
#valid_y = encoder.fit_transform(valid_y)
train_y[0:5]

array([  219,  1589,   450, 10192,  4689])